<a href="https://colab.research.google.com/github/Mushrifah/Secure-and-Private-AI-Challenge-/blob/master/Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [0]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [0]:
!pip install syft

In [0]:
import syft as sy

In [0]:
hook = sy.TorchHook(th)

In [8]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [10]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [13]:
bob._objects

{75883468801: tensor([1, 2, 3, 4, 5])}

In [14]:
x.location

<VirtualWorker id:bob #objects:1>

In [15]:
x.id_at_location

75883468801

In [16]:
x.id

67163333386

In [17]:
x.owner

<VirtualWorker id:me #objects:0>

In [18]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [19]:
x

(Wrapper)>[PointerTensor | me:67163333386 -> bob:75883468801]

In [20]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [21]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
# try this project here!

In [0]:
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr = x.send(bob, alice)

In [25]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob,alice)

In [27]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [29]:
x

(Wrapper)>[PointerTensor | me:81878126800 -> bob:33528075511]

In [33]:
y

(Wrapper)>[PointerTensor | me:15163315320 -> bob:36175331498]

In [0]:
z = x + y

In [35]:
z

(Wrapper)>[PointerTensor | me:55973330462 -> bob:45430956161]

In [36]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [37]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:89234732199 -> bob:4387497514]

In [38]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [45]:
z.backward()

(Wrapper)>[PointerTensor | me:47298963507 -> bob:14086994794]

In [0]:
x = x.get()

In [43]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [44]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
# try this project here!

In [0]:
input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [0]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [51]:
pred,target

((Wrapper)>[PointerTensor | me:90624355833 -> bob:45331258094],
 (Wrapper)>[PointerTensor | me:14145053680 -> bob:43877014087])

In [49]:
for i in range(10):

    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)

tensor(0.0058)
tensor(0.0037)
tensor(0.0024)
tensor(0.0015)
tensor(0.0010)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0001)


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects()

In [53]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [55]:
bob._objects

{85129183332: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [57]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [59]:
bob._objects

{87108394792: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [61]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [63]:
x

(Wrapper)>[PointerTensor | me:12018618202 -> bob:59117481675]

In [64]:
bob._objects

{59117481675: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [66]:
bob._objects

{59117481675: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [68]:
bob._objects

{59117481675: tensor([1, 2, 3, 4, 5])}

In [69]:
bob = bob.clear_objects()
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [72]:
bob._objects

{11538268902: tensor([1, 2, 3, 4, 5])}

In [0]:
#x = th.tensor([1,2,3,4,5]).send(bob)
#y = th.tensor([1,1,1,1,1])

In [0]:
#z = x + y

In [0]:
#x = th.tensor([1,2,3,4,5]).send(bob)
#y = th.tensor([1,1,1,1,1]).send(alice)

In [0]:
#z = x + y

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [81]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(3.5355)
tensor(1.2273)
tensor(0.7154)
tensor(0.4586)
tensor(0.2982)
tensor(0.1948)
tensor(0.1277)
tensor(0.0840)
tensor(0.0555)
tensor(0.0369)
tensor(0.0246)
tensor(0.0166)
tensor(0.0112)
tensor(0.0076)
tensor(0.0052)
tensor(0.0036)
tensor(0.0025)
tensor(0.0018)
tensor(0.0013)
tensor(0.0009)


In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
_data, _target=datasets[0]

In [90]:
_data.location

<VirtualWorker id:bob #objects:3>

In [0]:
model = model.send(_data.location)

In [94]:
list(model.parameters(model))

[Parameter containing:
 Parameter>[PointerTensor | me:78851398336 -> bob:92613965441],
 Parameter containing:
 Parameter>[PointerTensor | me:49460089199 -> bob:97732103113]]

In [0]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [86]:
train()

tensor(1.6828, requires_grad=True)
tensor(1.9857, requires_grad=True)
tensor(0.7072, requires_grad=True)
tensor(1.2496, requires_grad=True)
tensor(0.4175, requires_grad=True)
tensor(0.7796, requires_grad=True)
tensor(0.2561, requires_grad=True)
tensor(0.4909, requires_grad=True)
tensor(0.1599, requires_grad=True)
tensor(0.3126, requires_grad=True)
tensor(0.1015, requires_grad=True)
tensor(0.2015, requires_grad=True)
tensor(0.0657, requires_grad=True)
tensor(0.1314, requires_grad=True)
tensor(0.0434, requires_grad=True)
tensor(0.0869, requires_grad=True)
tensor(0.0292, requires_grad=True)
tensor(0.0581, requires_grad=True)
tensor(0.0200, requires_grad=True)
tensor(0.0394, requires_grad=True)
tensor(0.0139, requires_grad=True)
tensor(0.0270, requires_grad=True)
tensor(0.0099, requires_grad=True)
tensor(0.0187, requires_grad=True)
tensor(0.0071, requires_grad=True)
tensor(0.0131, requires_grad=True)
tensor(0.0051, requires_grad=True)
tensor(0.0092, requires_grad=True)
tensor(0.0037, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [95]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [98]:
bob._objects

{71111141777: tensor([1, 2, 3, 4, 5])}

In [100]:
alice._objects

{46366316757: (Wrapper)>[PointerTensor | alice:46366316757 -> bob:71111141777]}

In [0]:
y = x + x

In [102]:
y

(Wrapper)>[PointerTensor | me:73521135935 -> alice:53010498650]

In [103]:
bob._objects

{56388850573: tensor([ 2,  4,  6,  8, 10]),
 71111141777: tensor([1, 2, 3, 4, 5])}

In [104]:
alice._objects

{46366316757: (Wrapper)>[PointerTensor | alice:46366316757 -> bob:71111141777],
 53010498650: (Wrapper)>[PointerTensor | alice:53010498650 -> bob:56388850573]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [107]:
bob._objects

{9304464886: tensor([1, 2, 3, 4, 5])}

In [108]:
alice._objects

{49339262601: (Wrapper)>[PointerTensor | alice:49339262601 -> bob:9304464886]}

In [109]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:49339262601 -> bob:9304464886]

In [110]:
bob._objects

{9304464886: tensor([1, 2, 3, 4, 5])}

In [111]:
alice._objects

{}

In [112]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [113]:
bob._objects

{}

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [115]:
bob._objects

{86039290455: tensor([1, 2, 3, 4, 5])}

In [116]:
alice._objects

{45334471963: (Wrapper)>[PointerTensor | alice:45334471963 -> bob:86039290455]}

In [0]:
del x

In [118]:
bob._objects

{}

In [119]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [120]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [122]:
bob._objects

{25665671104: tensor([1, 2, 3, 4, 5])}

In [123]:
alice._objects

{}

In [124]:
x.move(alice)

(Wrapper)>[PointerTensor | me:53244380442 -> alice:53244380442]

In [125]:
bob._objects

{}

In [126]:
alice._objects

{53244380442: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [128]:
bob._objects

{82345393937: tensor([1, 2, 3, 4, 5])}

In [130]:
alice._objects

{50786070684: (Wrapper)>[PointerTensor | alice:50786070684 -> bob:82345393937],
 53244380442: tensor([1, 2, 3, 4, 5])}

In [131]:
x.remote_get()

(Wrapper)>[PointerTensor | me:28288705334 -> alice:50786070684]

In [132]:
bob._objects

{}

In [133]:
alice._objects

{50786070684: tensor([1, 2, 3, 4, 5]), 53244380442: tensor([1, 2, 3, 4, 5])}

In [134]:
x.move(bob)

(Wrapper)>[PointerTensor | me:28288705334 -> bob:28288705334]

In [135]:
x

(Wrapper)>[PointerTensor | me:28288705334 -> bob:28288705334]

In [136]:
bob._objects

{28288705334: tensor([1, 2, 3, 4, 5])}

In [137]:
alice._objects

{53244380442: tensor([1, 2, 3, 4, 5])}